# From Delta Lake to Amazon SageMaker

[Delta Lake](https://delta.io/) is a common open-source framework used for storing data in Lakehouse architectures.

In this sample we demonstrate how to integrate Delta Tables with Amazon SageMaker for performing data exploration, ingestion, processing, training, and hosting for Machine Learning.

---

## 1 - Data Exploration and Visualization

***Use Kernel: Data Science or Data Science 2.0 (Python 3) for running this notebook***

In this notebook, we will perform some Exploratory Data Analysis (EDA) over our Delta Tables with the two connection methods explained in the previous notebook.

<center><img src="../images/DeltaLake_to_SageMaker_0.png" width="60%"></center>


In [2]:
import sagemaker
sagemaker.__version__

'2.107.0'

In [3]:
import numpy as np
import pandas as pd
import boto3

In [4]:
# S3 bucket for saving processing job outputs
sm_session = sagemaker.Session()
bucket = sm_session.default_bucket()
region = sm_session.boto_region_name

sm_client = boto3.client('sagemaker')
iam_role = sagemaker.get_execution_role()

print('Default bucket: '+bucket)

Default bucket: sagemaker-eu-west-1-889960878219


----
### Option 1: Reading from a Delta Table stored in Amazon S3 via Spark Session and Context

In [6]:
# Import pyspark and build Spark session
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext

In [7]:
# Build list of packages entries using Maven coordinates (groupId:artifactId:version)
pkg_list = []
pkg_list.append("io.delta:delta-core_2.12:1.1.0")
pkg_list.append("org.apache.hadoop:hadoop-aws:3.2.2")

packages=(",".join(pkg_list))
print('packages: '+packages)

packages: io.delta:delta-core_2.12:1.1.0,org.apache.hadoop:hadoop-aws:3.2.2


In [8]:
# Instantiate Spark via builder
# Note: we use the `ContainerCredentialsProvider` to give us access to underlying IAM role permissions

spark = (SparkSession
    .builder
    .appName("PySparkApp") 
    .config("spark.jars.packages", packages) 
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
    .config("fs.s3a.aws.credentials.provider",'com.amazonaws.auth.ContainerCredentialsProvider') 
    .getOrCreate())

sc = spark.sparkContext

print('Spark version: '+str(sc.version))

:: loading settings :: url = jar:file:/opt/conda/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fb442fe9-413e-4efc-ae35-8a818023343e;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 714ms :: artifacts dl 36ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	--------

Spark version: 3.2.0


In [9]:
s3a_delta_table_uri=f's3a://{bucket}/delta_to_sagemaker/delta_format/'
print(s3a_delta_table_uri)

s3a://sagemaker-eu-west-1-889960878219/delta_to_sagemaker/delta_format/


In [10]:
# Create SQL command inserting the S3 path location

sql_cmd = f'SELECT * FROM delta.`{s3a_delta_table_uri}` ORDER BY medv'
print(f'SQL command: {sql_cmd}')

SQL command: SELECT * FROM delta.`s3a://sagemaker-eu-west-1-889960878219/delta_to_sagemaker/delta_format/` ORDER BY medv


In [11]:
# Execute SQL command which returns dataframe

sql_results = spark.sql(sql_cmd)
print(type(sql_results))

sql_results.show(10)

22/08/31 13:52:52 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


<class 'pyspark.sql.dataframe.DataFrame'>


+-------+---+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|   crim| zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+---+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|14.3337|  0| 18.1|   0|  0.7| 4.88| 100|1.5895| 24|666|   20.2|372.92|30.62|10.2|
|12.2472|  0| 18.1|   0|0.584|5.837|59.7|1.9976| 24|666|   20.2| 24.65|15.69|10.2|
|17.8667|  0| 18.1|   0|0.671|6.223| 100|1.3861| 24|666|   20.2|393.74|21.78|10.2|
|88.9762|  0| 18.1|   0|0.671|6.968|91.9|1.4165| 24|666|   20.2| 396.9|17.21|10.4|
|25.9406|  0| 18.1|   0|0.679|5.304|89.1|1.6475| 24|666|   20.2|127.36|26.64|10.4|
|22.0511|  0| 18.1|   0| 0.74|5.818|92.4|1.8662| 24|666|   20.2|391.45|22.11|10.5|
|24.3938|  0| 18.1|   0|  0.7|4.652| 100|1.4672| 24|666|   20.2| 396.9|28.28|10.5|
|12.8023|  0| 18.1|   0| 0.74|5.854|96.6|1.8956| 24|666|   20.2|240.52|23.79|10.8|
|15.8744|  0| 18.1|   0|0.671|6.545|99.1|1.5192| 24|666|   20.2| 396.9|21.08|10.9|
|37.

(TBC)..... steps for EDA with this Spark data frame

----
### Option 2: Reading from an external Delta Table via Delta Sharing

In [5]:
profile_file = 'https://raw.githubusercontent.com/delta-io/delta-sharing/main/examples/open-datasets.share'
!wget {profile_file} -P ./ -O 'open-datasets.share'

--2022-09-01 10:14:55--  https://raw.githubusercontent.com/delta-io/delta-sharing/main/examples/open-datasets.share
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 148 [text/plain]
Saving to: ‘open-datasets.share’

en-datasets.share   100%[===================>]     148  --.-KB/s    in 0s      

2022-09-01 10:14:55 (8.95 MB/s) - ‘open-datasets.share’ saved [148/148]



In [6]:
!cat ./open-datasets.share

{
  "shareCredentialsVersion": 1,
  "endpoint": "https://sharing.delta.io/delta-sharing/",
  "bearerToken": "faaie590d541265bcab1f2de9813274bf233"
}

In [7]:
sample_profile_file_url = sagemaker.Session().upload_data(
    './open-datasets.share', bucket=bucket, key_prefix='delta_to_sagemaker/delta_sharing/profile'
)

print(sample_profile_file_url)

s3://sagemaker-eu-west-1-889960878219/delta_to_sagemaker/delta_sharing/profile/open-datasets.share


In [8]:
# Create a SharingClient
import delta_sharing

client = delta_sharing.SharingClient(sample_profile_file_url)
table_url = profile_file + '#delta_sharing.default.boston-housing'

In [9]:
client.list_all_tables()

[Table(name='COVID_19_NYT', share='delta_sharing', schema='default'),
 Table(name='boston-housing', share='delta_sharing', schema='default'),
 Table(name='flight-asa_2008', share='delta_sharing', schema='default'),
 Table(name='lending_club', share='delta_sharing', schema='default'),
 Table(name='nyctaxi_2019', share='delta_sharing', schema='default'),
 Table(name='nyctaxi_2019_part', share='delta_sharing', schema='default'),
 Table(name='owid-covid-data', share='delta_sharing', schema='default')]

In [10]:
# Load the table as a Pandas DataFrame
print('Loading boston-housing table from Delta Lake')
train_data = delta_sharing.load_as_pandas(table_url)
print(f'Train data shape: {train_data.shape}')

Loading boston-housing table from Delta Lake
Train data shape: (506, 15)


In [11]:
train_data.head()

,ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
3,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
4,7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9


(TBC)... steps for EDA with this Pandas data-frame